# Explore Data Challenge

### Assumptions:
1. Data in tab-delimited format (fields separated by tabs)
1. Fields with quotes contain reserved characters such as `\t`, `\r`, and `\n`
1. Data is UTF-16LE encoded => convert to UTF-8
1. Data will have anomalies and will require judgment calls
1. Upload data to Redshift
1. Parallel algorithm: 
    * break up data into multiple parts (position, length) 
    * process the multiple parts, output each to separate TSV file
    * re-assemble multiple TSV files into a single TSV file

### Sections:
* Section 1 - explore data.tsv
* Section 2 - read non-anomalous data into pandas dataframe, explore
* Section 3 - experiment with algorithms to clean anomalous data

### References:
* Matthew Ropp (RingDNA) data challenge repo.  https://github.com/msropp/data-challenge
* Pandas-Redshift library repo.  https://github.com/agawronski/pandas_redshift

In [ ]:
# # install unidecode
# ! pip install unidecode

# # install pandas-redshift library if not already installed
# ! pip install pandas-redshift

In [ ]:
import os
from collections import Counter
import io

In [3]:
import boto3
import pandas as pd
import numpy as np
import pandas_redshift as pr

In [4]:
# pandas display settings
pd.set_option("display.max_columns", 999)
pd.set_option("display.max_rows", 999)

In [5]:
import unidecode
import re

### Section 1: Explore data

In [6]:
print(os.listdir(os.path.join('..', 'data')))

# create path to tsv file
path_to_tsv = os.path.join('..', 'data', 'data.tsv')

# read each line of file into a list, challenge documentation said 'utf-16-le' encoding
# https://realpython.com/read-write-files-python/
# https://stackoverflow.com/questions/4190683/python-string-replace-for-utf-16-le-file
with open(path_to_tsv, 'r', encoding='utf-16-le') as f:
    ls_lines_tsv_utf16le = f.readlines()

['data.tsv']


In [7]:
# remove accents on characters
# https://stackoverflow.com/questions/517923/what-is-the-best-way-to-remove-accents-normalize-in-a-python-unicode-string
# https://medium.com/@randombites/how-to-handle-accented-special-strings-175e65d96123
# https://stackoverflow.com/questions/31207287/converting-utf-16-to-utf-8
ls_lines_tsv = [unidecode.unidecode(x) for x in ls_lines_tsv_utf16le]

In [8]:
print(f'Number of lines in file: {len(ls_lines_tsv)}\n\n')

# print a few lines
for idx, each_line in enumerate(ls_lines_tsv[:5]):
    print(idx, each_line)

Number of lines in file: 1008


0 id	first_name	last_name	account_number	email

1 1	Addison	Marks	196296	ornare.lectus@et.edu

2 2	Dakota	Garza	409025	scelerisque@Praesentluctus.edu

3 3	Basia	Wolfe	637720	Aliquam@nullaIntegerurna.com

4 4	Germaine	Campbell	826846	id.magna@viverraMaecenas.ca



In [9]:
# challenge documentation says tab delimiters, same number of fields per line ...
# count number of tab delimiters per line
# https://www.programiz.com/python-programming/methods/string/count
ls_num_tabdelim = [x.count('\t') for x in ls_lines_tsv]

# what is frequency of num_tabs per line?
# https://stackoverflow.com/questions/2600191/how-can-i-count-the-occurrences-of-a-list-item
# https://www.w3schools.com/python/ref_func_sorted.asp
counter_numtabs = Counter(ls_num_tabdelim)
for x in sorted(counter_numtabs.items(), reverse=True):
    print(x)

(4, 995)
(3, 3)
(2, 5)
(1, 5)


In [10]:
def count_num_per_item(ls_input, str_input):
    '''
    Count number of occurrences per item in list.  
    Using this function to read a text file into a list and search for anomalies.
    Print the results, and output the counter-dictionary.
    
    For example, search the number of:
     * tabs per item string
     * newline characters per item string 
    
    Dependencies:
        from collections import Counter
        
    Input:
        ls_input - list, output of file.readlines()
        str_input - str, searching for substring
    Return:
        counter_num_per_item - Counterobject, looks like a dictionary, 
                                where key is number of occurances per item,
                                and value is number of items of this occurance
    '''
    
    # list of number of occurences per item
    ls_num_occurences_per_item = [x.count(str_input) for x in ls_input]
    
    # check frequency of occurences
    counter_num_per_item = Counter(ls_num_occurences_per_item)
    
    # print out frequency of occurences
    print(f'Total number of items in list: {len(ls_input)}\n')
    print(f'Frequency of occurences:')
    _ = [print(x) for x in sorted(counter_num_per_item.items(), reverse=True)]
    
    return counter_num_per_item

In [11]:
# not all of the lines have 4 tab-delimiters
_ = count_num_per_item(ls_lines_tsv, '\t')

Total number of items in list: 1008

Frequency of occurences:
(4, 995)
(3, 3)
(2, 5)
(1, 5)


In [12]:
# almost all lines have newline char
_ = count_num_per_item(ls_lines_tsv, "\n")

Total number of items in list: 1008

Frequency of occurences:
(1, 1007)
(0, 1)


In [13]:
# surprised none of lines have double-quote
_ = count_num_per_item(ls_lines_tsv, '\"')

Total number of items in list: 1008

Frequency of occurences:
(0, 1008)


In [14]:
# surprised none of lines have single-quote
_ = count_num_per_item(ls_lines_tsv, "\'")

Total number of items in list: 1008

Frequency of occurences:
(0, 1008)


In [15]:
# none of lines have carriage-return
_ = count_num_per_item(ls_lines_tsv, "\r")

Total number of items in list: 1008

Frequency of occurences:
(0, 1008)


In [16]:
# 6 of the lines have 1 dash
_ = count_num_per_item(ls_lines_tsv, "-")

Total number of items in list: 1008

Frequency of occurences:
(1, 6)
(0, 1002)


In [17]:
# one of the lines have 1 slash
_ = count_num_per_item(ls_lines_tsv, "/")

Total number of items in list: 1008

Frequency of occurences:
(1, 1)
(0, 1007)


In [18]:
# most items have 4 tab-delimters; anomalies don't have 4 tab-delimiters
# view lines with anomalies '\t', print out line number (0-index)
_ = [print(f'Line number {idx} => ', item) for idx, item in enumerate(ls_lines_tsv) if item.count('\t') != 4]

Line number 29 =>  29	Adena	Hobbs

Line number 30 =>  Bosley	656184

Line number 31 =>  	ac.ipsum.Phasellus@ut.net

Line number 84 =>  82	Jade	Battle

Line number 85 =>       	531695	lectus.justo@lorem.co.uk

Line number 220 =>  217	Boris

Line number 221 =>  Harrington	Harrington	325378	neque.Nullam.ut@laoreetlectus.edu

Line number 341 =>  337	NEHRU	MENDOZA	  859105

Line number 342 =>  	porttitor.interdum.Sed@Loremipsum.co.uk

Line number 780 =>  775	

Line number 781 =>  Barbara	Hurley	691210	enim.Mauris.quis@magna.net

Line number 991 =>  985	Cherokee	Indian

Line number 992 =>  	157172	enim@disparturient.edu



In [19]:
# view lines with anomalies on '\n' ... this is the last line in the data file 
_ = [print(f'Line number {idx} ... ', item) for idx, item in enumerate(ls_lines_tsv) if item.count('\n') != 1]

Line number 1007 ...  1000	Hermione	Morales	478506	eu.nulla@Donec.com


In [20]:
# view lines with anomalies on '-' ... 5 out of 6 are in the account_number, one is in the last_name
_ = [print(f'Line number {idx} ... ', item) for idx, item in enumerate(ls_lines_tsv) if item.count('-') > 0]

Line number 298 ...  294	Wyoming	Williams	454-586	odio.a.purus@Suspendisse.co.uk

Line number 392 ...  387	Palmer	Acevedo	168-722	ut.pellentesque.eget@eleifendCrassed.org

Line number 421 ...  416	elvis	RIVERA	865-008	Quisque.libero@vitaesodales.net

Line number 481 ...  476	Ivana	Chapman	357-130	et@Cras.co.uk

Line number 692 ...  687	Fay	Irwin	1181-61	metus@elitsedconsequat.com

Line number 961 ...  955	Veronica	Barr-Novel	953228	sollicitudin@Nunc.org



In [21]:
# view lines with anomalies on '/' ... one account number
_ = [print(f'Line number {idx} ... ', item) for idx, item in enumerate(ls_lines_tsv) if item.count('/') > 0]

Line number 315 ...  311	Kenneth	Bird	437/680	est@utpharetrased.org



### Observations in `Section 1`:
1. Total number of lines is 1008, which includes header line
1. Most of the lines have 4 tab-delimiters, although not true of 13 lines
1. The anomalies have fewer than 4 tab-delimiters because the data was split amongst multiple lines
1. One of the anomalous records has repeat last name 'Harrington'
1. Correcting the anomalies should be straightforward with a set of rules, but read the remaining lines into dataframe to examine

### Section 2: Read into dataframe (exception of anomalies w/o 4 tab-delimiters), and explore dataframe

In [22]:
# create list with only items with 4 tabs
ls_lines_tsv_4tabs = [x for x in ls_lines_tsv if x.count('\t')==4]

# convert list to string, already has newline character, no need to add to join
str_lines_tsv_4tabs = ''.join(ls_lines_tsv_4tabs)

# convert str to io.StringIO object so it can be read as CSV file
# https://www.kite.com/python/answers/how-to-create-a-pandas-dataframe-from-a-string-in-python
io_data_tsv = io.StringIO(str_lines_tsv_4tabs)

# create dataframe, tab-delimited
df_4_tabs = pd.read_csv(io_data_tsv, sep='\t')

print(f"Shape of dataframe: {df_4_tabs.shape}")
df_4_tabs.head()

Shape of dataframe: (994, 5)


,id,first_name,last_name,account_number,email
0,1,Addison,Marks,196296,ornare.lectus@et.edu
1,2,Dakota,Garza,409025,scelerisque@Praesentluctus.edu
2,3,Basia,Wolfe,637720,Aliquam@nullaIntegerurna.com
3,4,Germaine,Campbell,826846,id.magna@viverraMaecenas.ca
4,5,Lenore,Pennington,345284,aliquam@Integer.edu


In [23]:
print(df_4_tabs.nunique())

id                994
first_name        686
last_name         652
account_number    992
email             994
dtype: int64


In [24]:
# how many null values per column
print(df_4_tabs.isna().sum())

id                 0
first_name        12
last_name          9
account_number     0
email              0
dtype: int64


In [25]:
print(df_4_tabs.dtypes)

id                 int64
first_name        object
last_name         object
account_number    object
email             object
dtype: object


In [26]:
# is the account_number always numeric?
print('Number of records in dataframe: ', df_4_tabs.shape[0])
print('Number of records `account_number` is numeric: ', df_4_tabs['account_number'].str.isnumeric().sum())

Number of records in dataframe:  994
Number of records `account_number` is numeric:  976


In [27]:
# examine rows with null first_name or last_name
mask_null_name = (
    df_4_tabs['first_name'].isna()
    | df_4_tabs['last_name'].isna()
)
df_4_tabs.loc[mask_null_name]

,id,first_name,last_name,account_number,email
298,302,NaN,Copeland,547803,Vivamus.non.lorem@sed.ca
439,444,Roary,NaN,607318,mi.pede@orci.ca
514,519,NaN,Williamson,231131,Nullam.nisl@massa.ca
533,538,jana,NaN,824675,nisi.dictum.augue@nonenimcommodo.co.uk
547,552,Fitzgerald,NaN,317355,elit.a@velit.co.uk
561,566,Whoopi,NaN,125742,feugiat.non.lobortis@enimSednulla.net
596,601,NaN,Newton,601770,Duis.ac@nuncinterdum.edu
608,613,NaN,NaN,104969,dictum@Suspendisse.net
624,629,NaN,Riggs,242923,nulla.ante.iaculis@erat.net
668,673,Vielka,NaN,270462,metus.Aenean@lectusNullamsuscipit.edu


In [28]:
# create columns with flags on null names
df_4_tabs['null_firstname'] = df_4_tabs['first_name'].isna()
df_4_tabs['null_lastname'] = df_4_tabs['last_name'].isna()
df_4_tabs['null_name'] = df_4_tabs['null_firstname'] | df_4_tabs['null_lastname']

# create column with length of account_number
df_4_tabs['len_acct_num'] = df_4_tabs['account_number'].apply(len)

# create column with email domain name
# https://stackoverflow.com/questions/12504976/get-last-column-after-str-split-operation-on-column-in-pandas-dataframe
df_4_tabs['domain_name'] = df_4_tabs['email'].str.split('@').str[-1]

In [29]:
df_4_tabs.head()

,id,first_name,last_name,account_number,email,null_firstname,null_lastname,null_name,len_acct_num,domain_name
0,1,Addison,Marks,196296,ornare.lectus@et.edu,False,False,False,6,et.edu
1,2,Dakota,Garza,409025,scelerisque@Praesentluctus.edu,False,False,False,6,Praesentluctus.edu
2,3,Basia,Wolfe,637720,Aliquam@nullaIntegerurna.com,False,False,False,6,nullaIntegerurna.com
3,4,Germaine,Campbell,826846,id.magna@viverraMaecenas.ca,False,False,False,6,viverraMaecenas.ca
4,5,Lenore,Pennington,345284,aliquam@Integer.edu,False,False,False,6,Integer.edu


In [30]:
# examine each column for number of unique values ...
# ... look for .value_counts() situations where there are fewer than 10 counts per column
# print out number of unique values per column if > 10, print out .value_counts() if < 10
print(f'Shape of dataframe: {df_4_tabs.shape}\n')
print(f'Columns in dataframe: \n{list(df_4_tabs.columns)}\n')

for each_col in df_4_tabs.columns:
    num_counts = df_4_tabs[each_col].value_counts().shape[0]
    print(f'*** Number of value counts in column: {each_col} ===> {num_counts}\n')
    
    if num_counts < 10:
        print(f'Value counts of column: {each_col}')
        print(df_4_tabs[each_col].value_counts())
        print()

Shape of dataframe: (994, 10)

Columns in dataframe: 
['id', 'first_name', 'last_name', 'account_number', 'email', 'null_firstname', 'null_lastname', 'null_name', 'len_acct_num', 'domain_name']

*** Number of value counts in column: id ===> 994

*** Number of value counts in column: first_name ===> 686

*** Number of value counts in column: last_name ===> 652

*** Number of value counts in column: account_number ===> 992

*** Number of value counts in column: email ===> 994

*** Number of value counts in column: null_firstname ===> 2

Value counts of column: null_firstname
False    982
True      12
Name: null_firstname, dtype: int64

*** Number of value counts in column: null_lastname ===> 2

Value counts of column: null_lastname
False    985
True       9
Name: null_lastname, dtype: int64

*** Number of value counts in column: null_name ===> 2

Value counts of column: null_name
False    977
True      17
Name: null_name, dtype: int64

*** Number of value counts in column: len_acct_num =

In [31]:
# most values in account_number field are 6 characters long ... anomalies are not ...
# ... examine records where account number is not 6 digits long
df_4_tabs.query(" len_acct_num != 6 ")

,id,first_name,last_name,account_number,email,null_firstname,null_lastname,null_name,len_acct_num,domain_name
165,168,Nyssa,Livingston,897187,at@tortorat.net,False,False,False,7,tortorat.net
214,218,Ifeoma,Whitney,543699,Proin.nisl.sem@odiovelest.edu,False,False,False,7,odiovelest.edu
262,266,Howard,Cooley,167868,faucibus@ametultricies.org,False,False,False,7,ametultricies.org
272,276,Willa,Diaz,157615,Aliquam.tincidunt@metuseuerat.co.uk,False,False,False,7,metuseuerat.co.uk
290,294,Wyoming,Williams,454-586,odio.a.purus@Suspendisse.co.uk,False,False,False,7,Suspendisse.co.uk
295,299,Meghan,Whitaker,352355,leo@tempus.org,False,False,False,7,tempus.org
307,311,Kenneth,Bird,437/680,est@utpharetrased.org,False,False,False,7,utpharetrased.org
382,387,Palmer,Acevedo,168-722,ut.pellentesque.eget@eleifendCrassed.org,False,False,False,7,eleifendCrassed.org
411,416,elvis,RIVERA,865-008,Quisque.libero@vitaesodales.net,False,False,False,7,vitaesodales.net
471,476,Ivana,Chapman,357-130,et@Cras.co.uk,False,False,False,7,Cras.co.uk


In [32]:
# examine non-numeric account numbers by creating separate dataframe
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.isnumeric.html
mask_numeric_acct_num = df_4_tabs['account_number'].str.isnumeric()
df_4_tabs_nonnumeric_acct_num = df_4_tabs.loc[~mask_numeric_acct_num]

print('Show dataframe of non-numeric account numbers: \n')
display(df_4_tabs_nonnumeric_acct_num)
print()

print('Show the actual account_numbers: ')
print(df_4_tabs_nonnumeric_acct_num['account_number'].values)

Show dataframe of non-numeric account numbers: 



,id,first_name,last_name,account_number,email,null_firstname,null_lastname,null_name,len_acct_num,domain_name
165,168,Nyssa,Livingston,897187,at@tortorat.net,False,False,False,7,tortorat.net
214,218,Ifeoma,Whitney,543699,Proin.nisl.sem@odiovelest.edu,False,False,False,7,odiovelest.edu
262,266,Howard,Cooley,167868,faucibus@ametultricies.org,False,False,False,7,ametultricies.org
272,276,Willa,Diaz,157615,Aliquam.tincidunt@metuseuerat.co.uk,False,False,False,7,metuseuerat.co.uk
290,294,Wyoming,Williams,454-586,odio.a.purus@Suspendisse.co.uk,False,False,False,7,Suspendisse.co.uk
295,299,Meghan,Whitaker,352355,leo@tempus.org,False,False,False,7,tempus.org
307,311,Kenneth,Bird,437/680,est@utpharetrased.org,False,False,False,7,utpharetrased.org
382,387,Palmer,Acevedo,168-722,ut.pellentesque.eget@eleifendCrassed.org,False,False,False,7,eleifendCrassed.org
411,416,elvis,RIVERA,865-008,Quisque.libero@vitaesodales.net,False,False,False,7,vitaesodales.net
471,476,Ivana,Chapman,357-130,et@Cras.co.uk,False,False,False,7,Cras.co.uk



Show the actual account_numbers: 
[' 897187' '543699 ' '167868 ' ' 157615' '454-586' ' 352355' '437/680'
 '168-722' '865-008' '357-130' '  893425' ' 874570' '730368   ' ' 475426'
 '460962  ' '1181-61' ' 734417' '137599      ']


In [33]:
# reasons account_number non-numeric: leading/trailing whitespace, dash, slash ...
# ... clean up the account number, remove space and dashes and slashes => save as new column
df_4_tabs['clean_account_number'] = df_4_tabs['account_number'].str.replace(' ', '').str.replace('-', '').str.replace('/', '')

# in new clean column, any account numbers less than 6 digits? create new column to measure length
df_4_tabs['len_clean_acct_num'] = df_4_tabs['clean_account_number'].apply(len)

print('Number of characters in `clean_account_number` column: ')
print(df_4_tabs['len_clean_acct_num'].value_counts())
print()

print('Show the `clean_account_number` that is not 6 characters long: ')
print(df_4_tabs.query(" len_clean_acct_num != 6 ")['clean_account_number'].values)

Number of characters in `clean_account_number` column: 
6    992
5      2
Name: len_clean_acct_num, dtype: int64

Show the `clean_account_number` that is not 6 characters long: 
['93128' '75182']


In [34]:
# check for anomalies on first or last character of account number
df_4_tabs['first_char_clean_an'] = df_4_tabs['clean_account_number'].str[0]
df_4_tabs['last_char_clean_an'] = df_4_tabs['clean_account_number'].str[-1]

display(df_4_tabs['first_char_clean_an'].value_counts())
print()
display(df_4_tabs['last_char_clean_an'].value_counts())

2    120
3    120
1    114
4    113
9    113
7    112
5    109
8     97
6     96
Name: first_char_clean_an, dtype: int64

7    116
4    105
6    105
1    103
9     97
2     95
8     95
0     95
3     93
5     90
Name: last_char_clean_an, dtype: int64

In [35]:
# is the `clean_account_number` a unique primary key?  Or any duplicates?
# look for non-unique account numbers using .groupby().transform()
# https://pbpython.com/pandas_transform.html
df_4_tabs['nuniq_acct_num'] = df_4_tabs.groupby('clean_account_number')['clean_account_number'].transform(len)
df_4_tabs.query(" nuniq_acct_num>1 ")

,id,first_name,last_name,account_number,email,null_firstname,null_lastname,null_name,len_acct_num,domain_name,clean_account_number,len_clean_acct_num,first_char_clean_an,last_char_clean_an,nuniq_acct_num
410,415,Courtney,Salinas,114392,nec@nunc.net,False,False,False,6,nunc.net,114392,6,1,2,2
512,517,Wayne,Wilson,973758,nonummy@iaculis.co.uk,False,False,False,6,iaculis.co.uk,973758,6,9,8,2
899,905,Alice,Dyer,973758,Vivamus.nibh.dolor@gravidamauris.co.uk,False,False,False,6,gravidamauris.co.uk,973758,6,9,8,2
928,934,Nora,Leonard,114392,ornare.elit@libero.com,False,False,False,6,libero.com,114392,6,1,2,2


In [36]:
# look for account numbers (after cleaning) that are not length 6 characters
df_4_tabs.query(" len_clean_acct_num !=6 ")

,id,first_name,last_name,account_number,email,null_firstname,null_lastname,null_name,len_acct_num,domain_name,clean_account_number,len_clean_acct_num,first_char_clean_an,last_char_clean_an,nuniq_acct_num
830,836,Keegan,McCoy,93128,feugiat.nec@eu.co.uk,False,False,False,5,eu.co.uk,93128,5,9,8,1
954,960,Taylor,Wallace,75182,orci@dolor.net,False,False,False,5,dolor.net,75182,5,7,2,1


### Observations in `Section 2`:
1. The data with 4 tab-delimiters loads into dataframe easily
1. There are 994 rows, 5 columns (not counting header)
1. Some null values in first_name, last_name, or both
1. `id` column is numeric, but remaining columns convert to str (initially surprised `account_number` didn't convert numeric)
1. The `id` and `email` columns are completely unique, but the rest are not; even `account_number` has a few non-uniques
    * `account_number`: 114392
    * `account_number`: 973758
1. The `account_number` column has length of 6 characters in 974 records, but remaining have different lengths
1. After cleaning `account_number` by removing spaces, dashes, and slashes; two of the records are length 5 (consider leading zero that was removed?)
    * `account_number`: 93128
    * `account_number`: 75182
1. The first and last character in `account_number` appear random
1. Surprisingly, the `email` domains are almost unique, 904 uniques out of 994 rows

### Section 3: clean up tab-delimiters

In [37]:
# create list with anomalies '\t'
ls_lines_anom_tab_delim = [x for x in ls_lines_tsv if x.count('\t') != 4]
ls_lines_anom_tab_delim

['29\tAdena\tHobbs\n',
 'Bosley\t656184\n',
 '\tac.ipsum.Phasellus@ut.net\n',
 '82\tJade\tBattle\n',
 '     \t531695\tlectus.justo@lorem.co.uk\n',
 '217\tBoris\n',
 'Harrington\tHarrington\t325378\tneque.Nullam.ut@laoreetlectus.edu\n',
 '337\tNEHRU\tMENDOZA\t  859105\n',
 '\tporttitor.interdum.Sed@Loremipsum.co.uk\n',
 '775\t\n',
 'Barbara\tHurley\t691210\tenim.Mauris.quis@magna.net\n',
 '985\tCherokee\tIndian\n',
 '\t157172\tenim@disparturient.edu\n']

In [38]:
# create string with anomalies '\t'
str_anom_tab_delim = ''.join(ls_lines_anom_tab_delim)
str_anom_tab_delim

'29\tAdena\tHobbs\nBosley\t656184\n\tac.ipsum.Phasellus@ut.net\n82\tJade\tBattle\n     \t531695\tlectus.justo@lorem.co.uk\n217\tBoris\nHarrington\tHarrington\t325378\tneque.Nullam.ut@laoreetlectus.edu\n337\tNEHRU\tMENDOZA\t  859105\n\tporttitor.interdum.Sed@Loremipsum.co.uk\n775\t\nBarbara\tHurley\t691210\tenim.Mauris.quis@magna.net\n985\tCherokee\tIndian\n\t157172\tenim@disparturient.edu\n'

In [39]:
# clean string: (a) replace newline with tab, (b) remove whitespaces, (c) replace double-tab with single-tab
str_anom_tab_delim_clean = str_anom_tab_delim.replace('\n', '\t').replace(' ', '').replace('\t\t', '\t')
str_anom_tab_delim_clean

'29\tAdena\tHobbs\tBosley\t656184\tac.ipsum.Phasellus@ut.net\t82\tJade\tBattle\t531695\tlectus.justo@lorem.co.uk\t217\tBoris\tHarrington\tHarrington\t325378\tneque.Nullam.ut@laoreetlectus.edu\t337\tNEHRU\tMENDOZA\t859105\tporttitor.interdum.Sed@Loremipsum.co.uk\t775\tBarbara\tHurley\t691210\tenim.Mauris.quis@magna.net\t985\tCherokee\tIndian\t157172\tenim@disparturient.edu\t'

In [40]:
# examine string split on tab-delimeter
str_anom_tab_delim_clean.split('\t')

['29',
 'Adena',
 'Hobbs',
 'Bosley',
 '656184',
 'ac.ipsum.Phasellus@ut.net',
 '82',
 'Jade',
 'Battle',
 '531695',
 'lectus.justo@lorem.co.uk',
 '217',
 'Boris',
 'Harrington',
 'Harrington',
 '325378',
 'neque.Nullam.ut@laoreetlectus.edu',
 '337',
 'NEHRU',
 'MENDOZA',
 '859105',
 'porttitor.interdum.Sed@Loremipsum.co.uk',
 '775',
 'Barbara',
 'Hurley',
 '691210',
 'enim.Mauris.quis@magna.net',
 '985',
 'Cherokee',
 'Indian',
 '157172',
 'enim@disparturient.edu',
 '']

In [41]:
# convert list of text to str
# # create str that joins all records: (a) replace newline with tab, (b) remove whitespaces, (c) replace double-tab with single-tab
# str_tsv = ''.join(ls_lines_tsv).replace('\n', '\t').replace(' ', '').replace('\t\t', '\t')
ls_clean_tsv = [x.replace('\n', '\t').replace(' ', '').replace('\t\t', '\t') for x in ls_lines_tsv]
str_tsv = ''.join(ls_clean_tsv)

# look for characters not letters or numbers
print('Unique characters not numbers or letters:')
print(set(re.findall('[^a-zA-Z0-9]', str_tsv)))
print()

# look for characters not letters or numbers ... in the raw utf-16-le string
print('Unique characters not numbers or letters (utf-16-le):')
print(set(re.findall('[^a-zA-Z0-9]', ''.join(ls_lines_tsv_utf16le))))
print()

Unique characters not numbers or letters:
{'@', '_', '.', '-', '\t', '/'}

Unique characters not numbers or letters (utf-16-le):
{'@', '_', '\n', '.', '-', '\t', '/', 'È', ' '}



In [42]:
# use regex groups to identify records
# https://www.tutorialspoint.com/What-is-the-groups-method-in-regular-expressions-in-Python
'''
* id - one or more digits
* name - may include 0, 1, 2, 3 names
* account number - one or more digits that may contain '-' or '/'
* email may contain one or more '@', '.'
'''
r_str_match_0names = '(\d+\\t[0-9-/]*\\t[a-zA-Z@.]*)'
r_str_match_1names = '(\d+\\t[a-zA-Z]+\\t[0-9-/]*\\t[a-zA-Z@.]*)'
r_str_match_2names = '(\d+\\t[a-zA-Z]+\\t[a-zA-Z]+\\t[0-9-/]*\\t[a-zA-Z@.]*)'
r_str_match_3names = '(\d+\\t[a-zA-Z]+\\t[a-zA-Z]+\\t[a-zA-Z]+\\t[0-9-/]*\\t[a-zA-Z@.]*)'

# create list of records
ls_re_find_0names = re.findall(r_str_match_0names, str_anom_tab_delim_clean)
ls_re_find_1names = re.findall(r_str_match_1names, str_anom_tab_delim_clean)
ls_re_find_2names = re.findall(r_str_match_2names, str_anom_tab_delim_clean)
ls_re_find_3names = re.findall(r_str_match_3names, str_anom_tab_delim_clean)

print('Number of records with 0 names: ', len(ls_re_find_0names))
print('Number of records with 1 names: ', len(ls_re_find_1names))
print('Number of records with 2 names: ', len(ls_re_find_2names))
print('Number of records with 3 names: ', len(ls_re_find_3names))

Number of records with 0 names:  0
Number of records with 1 names:  0
Number of records with 2 names:  4
Number of records with 3 names:  2


In [43]:
print('Clean 3 names to 2 names: ')
print(ls_re_find_3names)
print()

# clean 3 names by removing middle name
for each_3name in ls_re_find_3names:
    
    # convert to list, split on tab delimiter
    temp_list = each_3name.split('\t')
    print(temp_list)
    
    # remove middle name
    del temp_list[2]
    print(temp_list)
    
    print()

Clean 3 names to 2 names: 
['29\tAdena\tHobbs\tBosley\t656184\tac.ipsum.Phasellus@ut.net', '217\tBoris\tHarrington\tHarrington\t325378\tneque.Nullam.ut@laoreetlectus.edu']

['29', 'Adena', 'Hobbs', 'Bosley', '656184', 'ac.ipsum.Phasellus@ut.net']
['29', 'Adena', 'Bosley', '656184', 'ac.ipsum.Phasellus@ut.net']

['217', 'Boris', 'Harrington', 'Harrington', '325378', 'neque.Nullam.ut@laoreetlectus.edu']
['217', 'Boris', 'Harrington', '325378', 'neque.Nullam.ut@laoreetlectus.edu']



In [44]:
# find id for any records that don't match 2 name pattern ...
# ... this is how I discovered the accented characters
ls_re_all_find_2names = re.findall(r_str_match_2names, str_tsv)
ls_id_regex = [x.split('\t')[0] for x in ls_re_all_find_2names]
for idx, item in enumerate(ls_id_regex):
    if idx == len(ls_id_regex)-1:
        print('End of List')
    elif int(item)+1 != int(ls_id_regex[idx+1]):
        print(item)

28
216
301
443
518
537
551
565
600
612
628
672
757
770
809
847
888
954
977
986
End of List


### Observations in `Section 3`:
1. Six records do not have 4 tab-delimiters
1. Many of the issues fall into 3 categories: (a) newline instead of tab, (b) extra whitespaces, (c) double tab-delimiter
    * Therefore correcting many of the mistakes by: (a) replace newline with tab, (b) remove whitespaces, (c) replace double-tab with single-tab
    * Join the list into a single string, then use regex to extract records
1. Two records continue to have issues because there are 3 names instead of 2 names
    * Correct these records by removing middle name
1. Using regex allowed me to detect accented characters, so I went back to correct in beginning using unidecode

### Algorithm Steps
1. Read the tsv file into list of strings
1. Remove records without 4 tab-delimiters => clean up so they will fit into dataframe (e.g. regex to remove middle names) => add back to list
1. Load entire list into pandas dataframe
1. Perform cleanup in pandas dataframe (mostly account_number, although minor issue of null names)
1. Export pandas dataframe to .tsv file

### Check AWS Cloud Resources

In [45]:
# create s3 resource
s3_resource = boto3.resource('s3')

# print available AWS S3 buckets
# https://stackoverflow.com/questions/49372761/boto3-using-boto3-resources3-to-list-all-s3-buckets
ls_buckets = [bucket.name for bucket in s3_resource.buckets.all()]
for idx, each_bucket in enumerate(ls_buckets):
    # print(idx, each_bucket)
    pass

Pandas Redshift | 2020-11-28 14:13:31,191 | botocore.credentials | INFO | Found credentials in shared credentials file: ~/.aws/credentials


In [46]:
# create redshift client
rs_client = boto3.client('redshift')

# print available redshift clusters
# https://stackoverflow.com/questions/34309151/get-list-clusters-amazon-redshift-using-python-with-boto3
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/redshift.html
di_rs_clusters = rs_client.describe_clusters()
print(di_rs_clusters['Clusters'])

[]
